In [1]:
import nltk, re, pprint
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

import pandas as pd
import numpy as np

import sys
import pymysql

ps = PorterStemmer()

In [ ]:
##Loading the file all at once
# df = pd.read_csv('recipes.txt', sep="*****eol*****", header=None)
# df.columns = ['URL', 'title', 'content']
# df['content'] = df['content'].str.replace(r"\<.*?\>"," ")
# df

In [13]:
f = open('contents.txt')

new_doc = 1
content = ""

for line in f.readlines():
        
    if new_doc:
        url, title  =  line.split(" ", 1)
        new_doc = 0
        print("URL: " , url)
        print("Title: " , title)
        continue

    if "*****eol*****" not in line:
        content += " " + line
    
    else:
        new_doc = 1
        #print("Content: " , content)
              
f.close()

URL:  п»їhttp://5000best.com/websites
Title:  5000 Best Websites 5000 Best Websites

URL:  http://5000best.com/websites/Books/
Title:  5000 Best Websites - Books 5000 Best Websites - Books

URL:  http://5000best.com/websites/8/
Title:  5000 Best Websites 5000 Best Websites

URL:  http://sharedcount.com/?url=http://blogger.com
Title:  SharedCount: Social URL Analytics SharedCount: Social URL AnalyticsPricingFAQAPI DocsContactLoginRegisterPricingFAQAPI DocsContactLoginRegisterSharedCountTrack URL Shares, Likes, and More.Or LoginRegisterTrusted by Over 1,000 Marketing Agencies WorldwideAbout UsCreated in 2010, SharedCount was the first API tool capable of giving you holistic engagement data on your website content. From simple use with WordPress, to larger API integrations, SharedCount makes it easy to see what content is connecting, what content is not, and where you should focus your efforts to maximize ROI.

URL:  http://5000best.com/websites/30/
Title:  5000 Best Websites 5000 Best We

In [15]:
### Laoding File into SQL ###
mydb = pymysql.connect("localhost","root","","web_crawl")
cursor = mydb.cursor()

# if tabel exists drop and create new one #### if its a big table do not do this
sql = "DROP TABLE IF EXISTS web_crawl"
cursor.execute(sql)
cursor.execute("CREATE TABLE web_crawl (URL VARCHAR(255), title VARCHAR(255), content VARCHAR(10000))")

# append to the table line by line
f = open('recipes.txt')
for line in f:
    
    URL, title, cont = line.split("|")
    sql = "INSERT INTO web_crawl (URL, title, content) VALUES (%s, %s, %s)"
    val = (URL, title, cont)
    cursor.execute(sql, val)
    mydb.commit()
    
f.close()

print("records inserted")
#######################################

records inserted


In [16]:
def token(text):
    
    # words = word_tokenize(text) # easy way out
    words = re.split('\W+', text)  # Spliting by white space and punctuation
    words = [word.lower() for word in words] # Case folding
    words = list(filter(None, words)) # Remove extra white space left behind 

    return words

In [17]:
def stopwords(words):
    
    stopwords = ['h1', 'div', 'class','h2', 'p', 'href', 'gel', 'pica', 'and' , 'a']
    filtered_words = []

    for w in words:
        if w not in stopwords:
            if w is not None:
                filtered_words.append(w)

    return filtered_words

In [18]:
def preprocess(raw_text):

    words = token(raw_text)
    filtered_words = stopwords(words)

    return filtered_words

In [35]:
cursor.execute("SELECT URL, content FROM web_crawl")
myresult = cursor.fetchall()

collection = []
docText = {}

for i in range(len(myresult)):

    doc = myresult[i][0]
    content = preprocess(myresult[i][1])
    
    collection.extend(content)
    docText[doc] = content